This crop is used to extract random bounding boxes that have no ground truth object

In [2]:
import os
import PIL
import numpy as np
import cv2
import pickle
from collections import namedtuple
from torchvision import transforms
from tqdm import tqdm as tqdm
import time

IndexEntry Objects are used to store all the information regarding a particular bounding box/ bounding box crop

In [3]:
IndexEntry = namedtuple('IndexEntry', ['img_path', 'sub_idx', 'classname', 'left', 'top', 'right', 'bottom'], verbose=False)
IndexEntryCrop = namedtuple('IndexEntryCrop', ['img_path', 'sub_idx', 'crop_path_1p0', 'classname', 'left', 'top', 'right', 'bottom'], verbose=False)

The following pickle file contains an list of IndexEntry Objects with the full information of the whole dataset

In [4]:
pickle_file = '/raid/user-data/lscheucher/projects/bounding_box_classifier/full_object_index.pickle'
with open(pickle_file, 'rb') as f:
    idxs = pickle.load(f)

Define the directory where the croped images should be written to

In [5]:
DATADIR = '/raid/user-data/lscheucher/tmp/pytorch_classifier_data'

In [7]:
idxs[0]

IndexEntry(img_path='/raid/group-data/uc150429/AID_DATA_201803/original_image/laidy_2018-10-18T10:01:37.401839.png', sub_idx=0, classname='Car', left=874, top=591, right=926, bottom=639)

In [ ]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    
    assert bb1['left'] <= bb1['right'] ,  str(bb1['left'])+","+str(bb1['right'])
    assert bb1['top'] <= bb1['bottom']
    assert bb2['left'] <= bb2['right']
    assert bb2['top'] <= bb2['bottom']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['left'], bb2['left'])
    y_top = max(bb1['top'], bb2['top'])
    x_right = min(bb1['right'], bb2['right'])
    y_bottom = min(bb1['bottom'], bb2['bottom'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['right'] - bb1['left']) * (bb1['bottom'] - bb1['top'])
    bb2_area = (bb2['right'] - bb2['left']) * (bb2['bottom'] - bb2['top'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
quantities = ['img_path', 'left', 'top','right', 'bottom']

In [9]:
imgname2bboxlist = {entry.img_path:[] for entry in idxs}

In [10]:
for entry in tqdm(idxs):
    imgname2bboxlist[entry.img_path].append({'left':entry.left, 'top':entry.top,'right':entry.right, 'bottom':entry.bottom})

100%|██████████| 484779/484779 [00:01<00:00, 450404.68it/s]


In [12]:
imgname2bboxlist[entry.img_path]

[{'bottom': 634, 'left': 607, 'right': 727, 'top': 598},
 {'bottom': 627, 'left': 1110, 'right': 1158, 'top': 595},
 {'bottom': 622, 'left': 1125, 'right': 1173, 'top': 590},
 {'bottom': 618, 'left': 1142, 'right': 1199, 'top': 582},
 {'bottom': 613, 'left': 1156, 'right': 1224, 'top': 572},
 {'bottom': 608, 'left': 1167, 'right': 1299, 'top': 548},
 {'bottom': 577, 'left': 1274, 'right': 1726, 'top': 401},
 {'bottom': 661, 'left': 895, 'right': 932, 'top': 621},
 {'bottom': 691, 'left': 745, 'right': 865, 'top': 603},
 {'bottom': 792, 'left': 833, 'right': 1117, 'top': 551},
 {'bottom': 658, 'left': 857, 'right': 897, 'top': 633}]

In [200]:
import random
def get_candidate():
    left = random.randint(0,1700)
    right = random.randint(left+10,min(1919,left+300))
    #print(left,right)
    top = random.randint(0,900)
    bottom = random.randint(top+int(0.3*(right-left)),min(1207,top+int(1.7*(right-left))))
    #print(top,bottom)
    #print((right-left)/(bottom-top))
    return {'left':left, 'top':top,'right':right, 'bottom':bottom}
def get_empty_bbox(bbox_list):
    candidates  = [get_candidate() for _ in range(100)]
    best_cand =None
    max_iou_all_cand = 1.0
    for cand in candidates:
        ###print(cand)
        maxiou_curcand = 1.0
        best_curcand = None
        for bbox in bbox_list:
            if get_iou(bbox, cand)<maxiou_curcand:
                maxiou=get_iou(bbox, cand)
                best_curcand = cand
                ###print(maxiou)
        if maxiou_curcand < max_iou_all_cand:
            best_cand = best_curcand

    return cand

In [191]:
img2emptybbox = {img_name:get_empty_bbox(bbox_list) for img_name, bbox_list in tqdm(imgname2bboxlist.items())}



  0%|          | 0/23386 [00:00<?, ?it/s]

  0%|          | 17/23386 [00:00<02:20, 165.80it/s]

  0%|          | 35/23386 [00:00<02:18, 168.87it/s]

  0%|          | 51/23386 [00:00<02:22, 163.34it/s]

  0%|          | 68/23386 [00:00<02:25, 160.81it/s]

  0%|          | 85/23386 [00:00<02:24, 160.71it/s]

  0%|          | 99/23386 [00:00<02:36, 149.04it/s]

  0%|          | 115/23386 [00:00<02:33, 151.20it/s]

  1%|          | 132/23386 [00:00<02:29, 155.22it/s]

  1%|          | 151/23386 [00:00<02:22, 163.30it/s]

  1%|          | 168/23386 [00:01<02:21, 163.58it/s]

  1%|          | 187/23386 [00:01<02:16, 170.36it/s]

  1%|          | 204/23386 [00:01<02:16, 170.17it/s]

  1%|          | 224/23386 [00:01<02:12, 174.79it/s]

  1%|          | 244/23386 [00:01<02:08, 179.99it/s]

  1%|          | 263/23386 [00:01<02:13, 172.82it/s]

  1%|          | 281/23386 [00:01<02:23, 161.41it/s]

  1%|▏         | 299/23386 [00:01<02:21, 163.53it/s]

  1%|▏         | 316/23386 [00:01<02:22, 16

 19%|█▉        | 4392/23386 [00:31<03:12, 98.47it/s] 

 19%|█▉        | 4406/23386 [00:31<02:58, 106.51it/s]

 19%|█▉        | 4422/23386 [00:31<02:41, 117.41it/s]

 19%|█▉        | 4438/23386 [00:31<02:32, 124.64it/s]

 19%|█▉        | 4452/23386 [00:31<02:28, 127.21it/s]

 19%|█▉        | 4466/23386 [00:31<02:29, 126.67it/s]

 19%|█▉        | 4480/23386 [00:31<02:27, 128.27it/s]

 19%|█▉        | 4494/23386 [00:31<02:30, 125.92it/s]

 19%|█▉        | 4511/23386 [00:31<02:19, 135.39it/s]

 19%|█▉        | 4527/23386 [00:32<02:13, 141.69it/s]

 19%|█▉        | 4545/23386 [00:32<02:07, 148.15it/s]

 20%|█▉        | 4564/23386 [00:32<01:58, 158.39it/s]

 20%|█▉        | 4581/23386 [00:32<02:03, 152.38it/s]

 20%|█▉        | 4601/23386 [00:32<01:55, 163.22it/s]

 20%|█▉        | 4618/23386 [00:32<02:03, 151.72it/s]

 20%|█▉        | 4634/23386 [00:32<02:04, 150.04it/s]

 20%|█▉        | 4650/23386 [00:32<02:11, 142.59it/s]

 20%|█▉        | 4665/23386 [00:32<02:17, 136.38it/s]

 20%|██   

 37%|███▋      | 8659/23386 [01:01<01:39, 148.64it/s]

 37%|███▋      | 8678/23386 [01:01<01:33, 157.70it/s]

 37%|███▋      | 8694/23386 [01:02<01:50, 133.38it/s]

 37%|███▋      | 8712/23386 [01:02<01:41, 143.94it/s]

 37%|███▋      | 8728/23386 [01:02<01:43, 141.35it/s]

 37%|███▋      | 8743/23386 [01:02<01:45, 138.43it/s]

 37%|███▋      | 8758/23386 [01:02<01:49, 133.32it/s]

 38%|███▊      | 8774/23386 [01:02<01:45, 138.15it/s]

 38%|███▊      | 8792/23386 [01:02<01:38, 148.08it/s]

 38%|███▊      | 8808/23386 [01:02<01:38, 147.89it/s]

 38%|███▊      | 8824/23386 [01:02<01:49, 133.29it/s]

 38%|███▊      | 8838/23386 [01:03<02:09, 112.23it/s]

 38%|███▊      | 8851/23386 [01:03<02:23, 101.09it/s]

 38%|███▊      | 8862/23386 [01:03<02:20, 103.58it/s]

 38%|███▊      | 8873/23386 [01:03<02:27, 98.25it/s] 

 38%|███▊      | 8886/23386 [01:03<02:19, 104.02it/s]

 38%|███▊      | 8897/23386 [01:03<02:19, 104.05it/s]

 38%|███▊      | 8915/23386 [01:03<02:02, 118.22it/s]

 38%|███▊ 

 54%|█████▍    | 12675/23386 [01:33<01:26, 123.86it/s]

 54%|█████▍    | 12693/23386 [01:33<01:18, 136.14it/s]

 54%|█████▍    | 12708/23386 [01:33<01:21, 131.64it/s]

 54%|█████▍    | 12724/23386 [01:33<01:17, 138.36it/s]

 54%|█████▍    | 12739/23386 [01:33<01:37, 109.54it/s]

 55%|█████▍    | 12752/23386 [01:33<01:43, 103.13it/s]

 55%|█████▍    | 12768/23386 [01:33<01:32, 115.24it/s]

 55%|█████▍    | 12781/23386 [01:33<01:29, 119.01it/s]

 55%|█████▍    | 12796/23386 [01:34<01:24, 125.18it/s]

 55%|█████▍    | 12813/23386 [01:34<01:18, 134.35it/s]

 55%|█████▍    | 12828/23386 [01:34<01:23, 127.01it/s]

 55%|█████▍    | 12842/23386 [01:34<01:25, 122.68it/s]

 55%|█████▍    | 12855/23386 [01:34<01:28, 118.63it/s]

 55%|█████▌    | 12870/23386 [01:34<01:23, 125.78it/s]

 55%|█████▌    | 12886/23386 [01:34<01:18, 134.05it/s]

 55%|█████▌    | 12903/23386 [01:34<01:15, 139.12it/s]

 55%|█████▌    | 12918/23386 [01:34<01:14, 140.30it/s]

 55%|█████▌    | 12934/23386 [01:35<01:12, 145.0

 72%|███████▏  | 16843/23386 [02:03<00:47, 138.03it/s]

 72%|███████▏  | 16859/23386 [02:03<00:46, 139.27it/s]

 72%|███████▏  | 16874/23386 [02:03<00:46, 140.97it/s]

 72%|███████▏  | 16890/23386 [02:03<00:44, 144.54it/s]

 72%|███████▏  | 16907/23386 [02:03<00:43, 149.81it/s]

 72%|███████▏  | 16924/23386 [02:04<00:42, 152.78it/s]

 72%|███████▏  | 16942/23386 [02:04<00:41, 156.84it/s]

 73%|███████▎  | 16958/23386 [02:04<00:41, 155.38it/s]

 73%|███████▎  | 16974/23386 [02:04<00:42, 151.95it/s]

 73%|███████▎  | 16990/23386 [02:04<00:42, 151.52it/s]

 73%|███████▎  | 17006/23386 [02:04<00:46, 136.87it/s]

 73%|███████▎  | 17020/23386 [02:04<00:50, 125.40it/s]

 73%|███████▎  | 17038/23386 [02:04<00:47, 134.16it/s]

 73%|███████▎  | 17052/23386 [02:04<00:49, 129.14it/s]

 73%|███████▎  | 17068/23386 [02:05<00:46, 135.10it/s]

 73%|███████▎  | 17083/23386 [02:05<00:45, 137.67it/s]

 73%|███████▎  | 17100/23386 [02:05<00:43, 145.07it/s]

 73%|███████▎  | 17115/23386 [02:05<00:44, 142.3

 90%|████████▉ | 21005/23386 [02:34<00:18, 128.53it/s]

 90%|████████▉ | 21019/23386 [02:34<00:19, 121.59it/s]

 90%|████████▉ | 21037/23386 [02:34<00:17, 134.21it/s]

 90%|█████████ | 21054/23386 [02:34<00:16, 141.83it/s]

 90%|█████████ | 21070/23386 [02:34<00:15, 145.41it/s]

 90%|█████████ | 21085/23386 [02:34<00:18, 124.94it/s]

 90%|█████████ | 21099/23386 [02:34<00:19, 118.78it/s]

 90%|█████████ | 21119/23386 [02:34<00:17, 132.97it/s]

 90%|█████████ | 21138/23386 [02:35<00:15, 143.88it/s]

 90%|█████████ | 21154/23386 [02:35<00:15, 146.10it/s]

 91%|█████████ | 21174/23386 [02:35<00:14, 157.84it/s]

 91%|█████████ | 21191/23386 [02:35<00:14, 156.39it/s]

 91%|█████████ | 21209/23386 [02:35<00:13, 158.89it/s]

 91%|█████████ | 21226/23386 [02:35<00:13, 154.52it/s]

 91%|█████████ | 21247/23386 [02:35<00:12, 166.11it/s]

 91%|█████████ | 21265/23386 [02:35<00:13, 157.38it/s]

 91%|█████████ | 21286/23386 [02:35<00:12, 167.59it/s]

 91%|█████████ | 21304/23386 [02:36<00:12, 162.6

TypeError: 'dict_items' object does not support indexing

In [199]:
for img_path, coords in tqdm(img2emptybbox.items()):
        img_final = np.zeros(shape=(224,224,3), dtype='uint8')
        img = cv2.imread(img_path)
        img = img[coords['top']:coords['bottom'], coords['left']:coords['right'], :]
        
        H, W, C = img.shape
        if H<=224 and W<=224:
            img=img

        elif H>W:
            fac = 224/H
            img = cv2.resize(img, (int(W*fac),224))
            H, W, C = img.shape
        else:#W>H
            fac = 224/W           # W    H
            img = cv2.resize(img, (224 ,int(H*fac)))
            H, W, C = img.shape
        dh = (224-H)>>1
        dw = (224-W)>>1
        img_final[dh:dh+H, dw:dw+W, :] = img

        filepath = img_path.split('/')[-1].split('.')[0]+str(entry.sub_idx)+'.png'
        filepath = os.path.join('/raid/user-data/lscheucher/tmp/pytorch_classifier_data/Empty/', filepath)
        cv2.imwrite(filepath, img_final)





  0%|          | 0/23386 [00:00<?, ?it/s]



  0%|          | 2/23386 [00:00<21:19, 18.28it/s]



  0%|          | 4/23386 [00:00<21:28, 18.15it/s]



  0%|          | 6/23386 [00:00<21:56, 17.76it/s]



  0%|          | 8/23386 [00:00<22:06, 17.63it/s]



  0%|          | 10/23386 [00:00<22:43, 17.14it/s]



  0%|          | 12/23386 [00:00<23:02, 16.90it/s]



  0%|          | 14/23386 [00:00<23:13, 16.77it/s]



  0%|          | 16/23386 [00:00<23:39, 16.46it/s]



  0%|          | 18/23386 [00:01<23:29, 16.58it/s]



  0%|          | 20/23386 [00:01<24:56, 15.61it/s]



  0%|          | 22/23386 [00:01<24:13, 16.08it/s]



  0%|          | 24/23386 [00:01<23:17, 16.71it/s]



  0%|          | 26/23386 [00:01<23:07, 16.83it/s]



  0%|          | 28/23386 [00:01<22:48, 17.07it/s]



  0%|          | 30/23386 [00:01<23:23, 16.64it/s]



  0%|          | 32/23386 [00:01<23:59, 16.22it/s]



  0%|          | 34/23386 [00:02<23:55, 16.26it/s]



  0%|          | 36/23386 [00:02<23:4

  2%|▏         | 490/23386 [00:29<22:27, 16.99it/s]



  2%|▏         | 492/23386 [00:29<22:20, 17.07it/s]



  2%|▏         | 494/23386 [00:29<22:27, 16.99it/s]



  2%|▏         | 496/23386 [00:29<23:06, 16.51it/s]



  2%|▏         | 498/23386 [00:29<22:58, 16.61it/s]



  2%|▏         | 500/23386 [00:29<22:30, 16.94it/s]



  2%|▏         | 502/23386 [00:29<22:18, 17.09it/s]



  2%|▏         | 504/23386 [00:29<22:32, 16.92it/s]



  2%|▏         | 506/23386 [00:30<22:46, 16.74it/s]



  2%|▏         | 508/23386 [00:30<22:32, 16.92it/s]



  2%|▏         | 510/23386 [00:30<22:27, 16.98it/s]



  2%|▏         | 512/23386 [00:30<22:26, 16.99it/s]



  2%|▏         | 514/23386 [00:30<22:44, 16.76it/s]



  2%|▏         | 516/23386 [00:30<22:30, 16.94it/s]



  2%|▏         | 518/23386 [00:30<22:58, 16.59it/s]



  2%|▏         | 520/23386 [00:30<23:30, 16.21it/s]



  2%|▏         | 522/23386 [00:31<23:36, 16.14it/s]



  2%|▏         | 524/23386 [00:31<22:55, 16.62it/s]



  2%|▏    

  4%|▍         | 978/23386 [00:58<22:31, 16.58it/s]



  4%|▍         | 980/23386 [00:58<22:15, 16.77it/s]



  4%|▍         | 982/23386 [00:58<22:13, 16.81it/s]



  4%|▍         | 984/23386 [00:58<22:01, 16.95it/s]



  4%|▍         | 986/23386 [00:58<22:18, 16.73it/s]



  4%|▍         | 988/23386 [00:59<22:54, 16.29it/s]



  4%|▍         | 990/23386 [00:59<22:52, 16.32it/s]



  4%|▍         | 992/23386 [00:59<22:25, 16.64it/s]



  4%|▍         | 994/23386 [00:59<21:45, 17.15it/s]



  4%|▍         | 996/23386 [00:59<21:45, 17.15it/s]



  4%|▍         | 998/23386 [00:59<21:50, 17.09it/s]



  4%|▍         | 1000/23386 [00:59<22:06, 16.87it/s]



  4%|▍         | 1002/23386 [00:59<21:59, 16.96it/s]



  4%|▍         | 1004/23386 [00:59<21:40, 17.21it/s]



  4%|▍         | 1006/23386 [01:00<22:04, 16.90it/s]



  4%|▍         | 1008/23386 [01:00<22:06, 16.87it/s]



  4%|▍         | 1010/23386 [01:00<22:40, 16.45it/s]



  4%|▍         | 1012/23386 [01:00<22:45, 16.39it/s]



  4

  6%|▌         | 1458/23386 [01:27<21:59, 16.62it/s]



  6%|▌         | 1460/23386 [01:27<21:31, 16.98it/s]



  6%|▋         | 1462/23386 [01:27<21:53, 16.69it/s]



  6%|▋         | 1464/23386 [01:27<22:34, 16.18it/s]



  6%|▋         | 1466/23386 [01:27<22:40, 16.11it/s]



  6%|▋         | 1468/23386 [01:27<22:43, 16.07it/s]



  6%|▋         | 1470/23386 [01:27<22:19, 16.37it/s]



  6%|▋         | 1472/23386 [01:28<22:13, 16.44it/s]



  6%|▋         | 1474/23386 [01:28<22:16, 16.39it/s]



  6%|▋         | 1476/23386 [01:28<22:43, 16.07it/s]



  6%|▋         | 1478/23386 [01:28<23:01, 15.85it/s]



  6%|▋         | 1480/23386 [01:28<22:18, 16.36it/s]



  6%|▋         | 1482/23386 [01:28<21:35, 16.91it/s]



  6%|▋         | 1484/23386 [01:28<21:48, 16.73it/s]



  6%|▋         | 1486/23386 [01:28<21:49, 16.73it/s]



  6%|▋         | 1488/23386 [01:29<21:48, 16.74it/s]



  6%|▋         | 1490/23386 [01:29<21:44, 16.78it/s]



  6%|▋         | 1492/23386 [01:29<21:31, 16.95i

  8%|▊         | 1938/23386 [01:58<21:16, 16.80it/s]



  8%|▊         | 1940/23386 [01:58<21:28, 16.64it/s]



  8%|▊         | 1942/23386 [01:58<21:56, 16.29it/s]



  8%|▊         | 1944/23386 [01:59<22:03, 16.21it/s]



  8%|▊         | 1946/23386 [01:59<21:46, 16.42it/s]



  8%|▊         | 1948/23386 [01:59<21:36, 16.54it/s]



  8%|▊         | 1950/23386 [01:59<21:51, 16.35it/s]



  8%|▊         | 1952/23386 [01:59<21:56, 16.28it/s]



  8%|▊         | 1954/23386 [01:59<21:50, 16.35it/s]



  8%|▊         | 1956/23386 [01:59<21:55, 16.29it/s]



  8%|▊         | 1958/23386 [01:59<22:05, 16.16it/s]



  8%|▊         | 1960/23386 [02:00<21:34, 16.56it/s]



  8%|▊         | 1962/23386 [02:00<22:01, 16.21it/s]



  8%|▊         | 1964/23386 [02:00<22:08, 16.13it/s]



  8%|▊         | 1966/23386 [02:00<22:07, 16.14it/s]



  8%|▊         | 1968/23386 [02:00<22:10, 16.10it/s]



  8%|▊         | 1970/23386 [02:00<22:42, 15.72it/s]



  8%|▊         | 1972/23386 [02:00<22:23, 15.94i

...

 92%|█████████▏| 21423/23386 [24:39<02:29, 13.15it/s]



 92%|█████████▏| 21425/23386 [24:39<02:26, 13.40it/s]



 92%|█████████▏| 21427/23386 [24:39<02:22, 13.78it/s]



 92%|█████████▏| 21429/23386 [24:39<02:20, 13.94it/s]



 92%|█████████▏| 21431/23386 [24:39<02:19, 14.03it/s]



 92%|█████████▏| 21433/23386 [24:39<02:21, 13.82it/s]



 92%|█████████▏| 21435/23386 [24:40<02:18, 14.07it/s]



 92%|█████████▏| 21437/23386 [24:40<02:15, 14.37it/s]



 92%|█████████▏| 21439/23386 [24:40<02:16, 14.29it/s]



 92%|█████████▏| 21441/23386 [24:40<02:14, 14.45it/s]



 92%|█████████▏| 21443/23386 [24:40<02:14, 14.47it/s]



 92%|█████████▏| 21445/23386 [24:40<02:12, 14.64it/s]



 92%|█████████▏| 21447/23386 [24:40<02:11, 14.70it/s]



 92%|█████████▏| 21449/23386 [24:41<02:11, 14.77it/s]



 92%|█████████▏| 21451/23386 [24:41<02:13, 14.51it/s]



 92%|█████████▏| 21453/23386 [24:41<02:15, 14.29it/s]



 92%|█████████▏| 21455/23386 [24:41<02:18, 13.98it/s]



 92%|█████████▏| 21457/23386 [2

 94%|█████████▎| 21895/23386 [25:12<01:44, 14.28it/s]



 94%|█████████▎| 21897/23386 [25:12<01:44, 14.29it/s]



 94%|█████████▎| 21899/23386 [25:12<01:42, 14.47it/s]



 94%|█████████▎| 21901/23386 [25:12<01:41, 14.60it/s]



 94%|█████████▎| 21903/23386 [25:13<01:42, 14.49it/s]



 94%|█████████▎| 21905/23386 [25:13<01:43, 14.26it/s]



 94%|█████████▎| 21907/23386 [25:13<01:42, 14.48it/s]



 94%|█████████▎| 21909/23386 [25:13<01:42, 14.44it/s]



 94%|█████████▎| 21911/23386 [25:13<01:42, 14.33it/s]



 94%|█████████▎| 21913/23386 [25:13<01:42, 14.31it/s]



 94%|█████████▎| 21915/23386 [25:13<01:42, 14.35it/s]



 94%|█████████▎| 21917/23386 [25:14<01:41, 14.42it/s]



 94%|█████████▎| 21919/23386 [25:14<01:42, 14.38it/s]



 94%|█████████▎| 21921/23386 [25:14<01:42, 14.25it/s]



 94%|█████████▎| 21923/23386 [25:14<01:45, 13.92it/s]



 94%|█████████▍| 21925/23386 [25:14<01:44, 14.00it/s]



 94%|█████████▍| 21927/23386 [25:14<01:42, 14.27it/s]



 94%|█████████▍| 21929/23386 [2

 96%|█████████▌| 22367/23386 [25:45<01:21, 12.53it/s]



 96%|█████████▌| 22369/23386 [25:45<01:18, 12.93it/s]



 96%|█████████▌| 22371/23386 [25:45<01:15, 13.41it/s]



 96%|█████████▌| 22373/23386 [25:45<01:14, 13.64it/s]



 96%|█████████▌| 22375/23386 [25:45<01:13, 13.68it/s]



 96%|█████████▌| 22377/23386 [25:46<01:12, 13.85it/s]



 96%|█████████▌| 22379/23386 [25:46<01:11, 14.09it/s]



 96%|█████████▌| 22381/23386 [25:46<01:10, 14.19it/s]



 96%|█████████▌| 22383/23386 [25:46<01:10, 14.14it/s]



 96%|█████████▌| 22385/23386 [25:46<01:09, 14.33it/s]



 96%|█████████▌| 22387/23386 [25:46<01:09, 14.34it/s]



 96%|█████████▌| 22389/23386 [25:46<01:09, 14.40it/s]



 96%|█████████▌| 22391/23386 [25:47<01:08, 14.57it/s]



 96%|█████████▌| 22393/23386 [25:47<01:08, 14.49it/s]



 96%|█████████▌| 22395/23386 [25:47<01:09, 14.21it/s]



 96%|█████████▌| 22397/23386 [25:47<01:09, 14.28it/s]



 96%|█████████▌| 22399/23386 [25:47<01:08, 14.48it/s]



 96%|█████████▌| 22401/23386 [2

 98%|█████████▊| 22839/23386 [26:18<00:38, 14.38it/s]



 98%|█████████▊| 22841/23386 [26:18<00:37, 14.48it/s]



 98%|█████████▊| 22843/23386 [26:19<00:36, 14.71it/s]



 98%|█████████▊| 22845/23386 [26:19<00:36, 14.65it/s]



 98%|█████████▊| 22847/23386 [26:19<00:37, 14.50it/s]



 98%|█████████▊| 22849/23386 [26:19<00:37, 14.18it/s]



 98%|█████████▊| 22851/23386 [26:19<00:38, 14.07it/s]



 98%|█████████▊| 22853/23386 [26:19<00:38, 13.96it/s]



 98%|█████████▊| 22855/23386 [26:19<00:37, 14.09it/s]



 98%|█████████▊| 22857/23386 [26:20<00:37, 14.18it/s]



 98%|█████████▊| 22859/23386 [26:20<00:36, 14.33it/s]



 98%|█████████▊| 22861/23386 [26:20<00:36, 14.35it/s]



 98%|█████████▊| 22863/23386 [26:20<00:36, 14.21it/s]



 98%|█████████▊| 22865/23386 [26:20<00:36, 14.44it/s]



 98%|█████████▊| 22867/23386 [26:20<00:36, 14.37it/s]



 98%|█████████▊| 22869/23386 [26:20<00:36, 14.16it/s]



 98%|█████████▊| 22871/23386 [26:21<00:36, 14.20it/s]



 98%|█████████▊| 22873/23386 [2

100%|█████████▉| 23311/23386 [26:52<00:05, 14.46it/s]



100%|█████████▉| 23313/23386 [26:52<00:05, 14.60it/s]



100%|█████████▉| 23315/23386 [26:52<00:04, 14.61it/s]



100%|█████████▉| 23317/23386 [26:52<00:04, 14.37it/s]



100%|█████████▉| 23319/23386 [26:53<00:04, 14.20it/s]



100%|█████████▉| 23321/23386 [26:53<00:04, 14.05it/s]



100%|█████████▉| 23323/23386 [26:53<00:04, 14.21it/s]



100%|█████████▉| 23325/23386 [26:53<00:04, 14.23it/s]



100%|█████████▉| 23327/23386 [26:53<00:04, 14.35it/s]



100%|█████████▉| 23329/23386 [26:53<00:03, 14.35it/s]



100%|█████████▉| 23331/23386 [26:53<00:03, 14.54it/s]



100%|█████████▉| 23333/23386 [26:54<00:03, 14.63it/s]



100%|█████████▉| 23335/23386 [26:54<00:03, 14.68it/s]



100%|█████████▉| 23337/23386 [26:54<00:03, 14.82it/s]



100%|█████████▉| 23339/23386 [26:54<00:03, 14.69it/s]



100%|█████████▉| 23341/23386 [26:54<00:03, 14.63it/s]



100%|█████████▉| 23343/23386 [26:54<00:02, 14.54it/s]



100%|█████████▉| 23345/23386 [2